In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#i need that data that contaons just 3's and 7's
#that is my first step
#importing all i need to make my life easy


#hide
from fastai.vision.all import *
#from fastbook import *

matplotlib.rc('image', cmap='Greys')

In [3]:
path = untar_data(URLs.MNIST_SAMPLE)


In [4]:
path.ls()

(#3) [Path('/root/.fastai/data/mnist_sample/valid'),Path('/root/.fastai/data/mnist_sample/labels.csv'),Path('/root/.fastai/data/mnist_sample/train')]

In [5]:
(path/'train').ls()


(#2) [Path('/root/.fastai/data/mnist_sample/train/7'),Path('/root/.fastai/data/mnist_sample/train/3')]

In [6]:
threes = (path/'train'/'3').ls().sorted()
sevens = (path/'train'/'7').ls().sorted()
threes

(#6131) [Path('/root/.fastai/data/mnist_sample/train/3/10.png'),Path('/root/.fastai/data/mnist_sample/train/3/10000.png'),Path('/root/.fastai/data/mnist_sample/train/3/10011.png'),Path('/root/.fastai/data/mnist_sample/train/3/10031.png'),Path('/root/.fastai/data/mnist_sample/train/3/10034.png'),Path('/root/.fastai/data/mnist_sample/train/3/10042.png'),Path('/root/.fastai/data/mnist_sample/train/3/10052.png'),Path('/root/.fastai/data/mnist_sample/train/3/1007.png'),Path('/root/.fastai/data/mnist_sample/train/3/10074.png'),Path('/root/.fastai/data/mnist_sample/train/3/10091.png'),Path('/root/.fastai/data/mnist_sample/train/3/10093.png'),Path('/root/.fastai/data/mnist_sample/train/3/10097.png'),Path('/root/.fastai/data/mnist_sample/train/3/10099.png'),Path('/root/.fastai/data/mnist_sample/train/3/10116.png'),Path('/root/.fastai/data/mnist_sample/train/3/10125.png'),Path('/root/.fastai/data/mnist_sample/train/3/10137.png'),Path('/root/.fastai/data/mnist_sample/train/3/10141.png'),Path('/ro

In [9]:
seven_tensors = [tensor(Image.open(o)) for o in sevens]
three_tensors = [tensor(Image.open(o)) for o in threes]
len(three_tensors),len(seven_tensors)

(6131, 6265)

In [12]:
stacked_threes = torch.stack(three_tensors).float()/255
stacked_seven = torch.stack(seven_tensors).float()/255

In [17]:
train_x = torch.cat([stacked_threes, stacked_seven]).view(-1, 28*28)
train_x.shape

torch.Size([12396, 784])

In [20]:
train_y = tensor([1]*len(three_tensors) + [0]*len(seven_tensors)).unsqueeze(1)
train_y.shape

torch.Size([12396, 1])

In [22]:
ds = list(zip(train_x, train_y))
x, y = ds[0]
x.shape, y

(torch.Size([784]), tensor([1]))

In [25]:
df = DataLoader(ds, batch_size = 256)
xb,yb = first(df)
xb.shape,yb.shape

(torch.Size([256, 784]), torch.Size([256, 1]))

In [26]:

valid_3_tens = torch.stack([tensor(Image.open(o))
                            for o in (path/'valid'/'3').ls()])
valid_3_tens = valid_3_tens.float()/255
valid_7_tens = torch.stack([tensor(Image.open(o))
                            for o in (path/'valid'/'7').ls()])
valid_7_tens = valid_7_tens.float()/255
valid_3_tens.shape,valid_7_tens.shape


(torch.Size([1010, 28, 28]), torch.Size([1028, 28, 28]))

In [27]:
valid_x = torch.cat([valid_3_tens, valid_7_tens]).view(-1, 28*28)
valid_y = tensor([1]*len(valid_3_tens) + [0]*len(valid_7_tens)).unsqueeze(1)
valid_dset = list(zip(valid_x,valid_y))

In [28]:
dl = DataLoader(ds, batch_size=256)
xb,yb = first(dl)
xb.shape,yb.shape

(torch.Size([256, 784]), torch.Size([256, 1]))

In [43]:
valid_dl = DataLoader(valid_dset, batch_size=256)

In [29]:
def init_params(size, std=1.0):
    return (torch.randn(size)*std).requires_grad_()
    

In [30]:
weights = init_params(28*28, 1)
bias = init_params(1)

In [31]:
weights.shape

torch.Size([784])

In [33]:
#just for testing
preds = (train_x[0] * weights).sum() + bias 
preds

tensor([0.6127], grad_fn=<AddBackward0>)

In [39]:
preds = (train_x[1] * weights).sum() + bias 
preds

tensor([-2.9322], grad_fn=<AddBackward0>)

In [34]:
#train with batch size =4 
batch = train_x[:4]


__STEP 1: PREDICTION FUNCTION__

In [60]:
def linear1(xb):
    preds = xb@weights + bias 
    return preds

__STEP 2 : LOSS FUNCTION__

In [61]:
def mnist_loss(preds, trgts):
    preds = preds.sigmoid()
    return torch.where(trgts==1.0, 1-preds, preds).mean()

__STEP 3 : CALCULATE GRADIENTS__

In [62]:
def calc_grad(xb, yb, model):
    preds = model(xb)
    loss= mnist_loss(preds, yb)
    loss.backward()

__STEP 4 : FIND ACCURACY__

In [71]:
def batch_accuracy(xb, yb):
    preds = xb.sigmoid()
    correct = (preds >0.5).float() == yb
    return correct.float().mean()
    
    

In [72]:
def validate_epoch(model):
    accs = [batch_accuracy(model(xb), yb) for xb, yb in valid_dl]
    return round(torch.stack(accs).mean().item(), 4)
    

__STEP 5 : TRAIN AN EPOCH__

In [73]:
def train_epoch(model, lr, params):
    for xb, yb in df:
        calc_grad(xb, yb, model)
        for p in params:
            p.data -= p.grad * lr 
            p.grad.zero_()
    

In [74]:
lr = 1
params = weights, bias
train_epoch(linear1, lr, params)
validate_epoch(linear1)

0.8767

In [75]:
for i in range(20):
    train_epoch(linear1, lr, params)
    print(validate_epoch(linear1), end= ' ')

0.9209 0.9384 0.9428 0.9471 0.9508 0.9523 0.9546 0.956 0.959 0.9595 0.96 0.96 0.9614 0.9619 0.9619 0.9633 0.9643 0.9648 0.9652 0.9647 